In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
#Loading MNIST Dataset as a substitute for USPS
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#Resize the 16x16 and normalize to [-1,1]
x_train = tf.image.resize(x_train[..., np.newaxis], [16,16]).numpy() / 127.5 - 1
x_test = tf.image.resize(x_test[..., np.newaxis], [16,16]).numpy() / 127.5 - 1

#One hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [5]:
model = models.Sequential([
    layers.Conv2D(6, (5, 5), activation='relu', input_shape=(16, 16, 1), padding='same'),
    layers.AveragePooling2D((2, 2)),
    layers.Conv2D(12, (5, 5), activation='relu', padding='same'),
    layers.AveragePooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/opt/anaconda3/envs/research/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8024 - loss: 0.6380 - val_accuracy: 0.9626 - val_loss: 0.1148
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9617 - loss: 0.1231 - val_accuracy: 0.9741 - val_loss: 0.0801
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9727 - loss: 0.0854 - val_accuracy: 0.9775 - val_loss: 0.0752
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9777 - loss: 0.0719 - val_accuracy: 0.9833 - val_loss: 0.0515
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9812 - loss: 0.0587 - val_accuracy: 0.9826 - val_loss: 0.0577
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9836 - loss: 0.0517 - val_accuracy: 0.9842 - val_loss: 0.0507
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9863 - loss: 0.0425 - val_accuracy: 0.9870 - val_loss: 0.0456
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9873 - loss: 0.0399 - 

In [9]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Implement a simple rejection mechanism based on confidence thresholds
def predict_with_rejection(model, data, threshold=0.9):
    predictions = model.predict(data)
    confident_predictions = np.max(predictions, axis=1) > threshold
    rejected = np.sum(~confident_predictions)
    accepted_accuracy = np.mean(np.argmax(predictions[confident_predictions], axis=1) == np.argmax(y_test[confident_predictions], axis=1))
    return accepted_accuracy, rejected

accepted_accuracy, rejected = predict_with_rejection(model, x_test, threshold=0.9)
print(f"Accepted Accuracy: {accepted_accuracy * 100:.2f}%, Rejected: {rejected} samples")


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9837 - loss: 0.0463  
Test Accuracy: 98.71%
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step
Accepted Accuracy: 99.68%, Rejected: 301 samples


In [11]:
# Save the model for deployment
model.save('digit_recognition_model.h5')

# For deployment, convert to a lightweight format (e.g., TensorFlow Lite)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('digit_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved and ready for deployment.")


INFO:tensorflow:Assets written to: /var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6/assets


INFO:tensorflow:Assets written to: /var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6/assets


Saved artifact at '/var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 16, 16, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  5279360848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279361616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279359504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279359888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279362384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279360464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279362000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279361424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279362192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5279363536: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1737775259.973394 5517314 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.


Model saved and ready for deployment.


W0000 00:00:1737775259.973740 5517314 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-25 04:20:59.975303: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6
2025-01-25 04:20:59.975802: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-25 04:20:59.975808: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6
I0000 00:00:1737775259.980366 5517314 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-01-25 04:20:59.981063: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-25 04:21:00.010648: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/pc/_hmwmdzj5vg861_7lwm88pzw0000gn/T/tmpukbrjqc6
2025-01-25 04:21:00.018617: I tensorflow/cc/s